# Face_Mask_detection

Introduction:

In today's world, the importance of face mask detection cannot be overstated. To address this pressing concern, this notebook harnesses the power of the TensorFlow 2 Object Detection API to train a state-of-the-art model. Specifically, we explore the training of an SSD-MobileNet or an EfficientDet model using a custom dataset. The end goal of this project is to not only train a highly accurate model but also to convert it to TensorFlow Lite format. Through this Colab, you will embark on a journey to create and obtain a TensorFlow Lite model, which can be seamlessly deployed on your personal computer, Android phone, or even on compact edge devices like the Raspberry Pi. This project empowers you to contribute to public health and safety by detecting the presence of face masks with precision and convenience.

I obtained my dataset from Kaggle, which comprises images depicting individuals both with and without face masks. Now, let's move on to the crucial step of gathering and labeling the training images. For a preliminary model to prove the concept, we recommend starting with approximately 7k images. These training images should exhibit a mix of random objects alongside the objects of interest, i.e., faces with and without masks. Furthermore, it's important to ensure that the images encompass a diverse range of backgrounds and lighting conditions to enhance the model's robustness.

For comprehensive guidance on the process of gathering and labeling images for training an object detection model, please watch the instructional YouTube video provided below, which offers valuable insights and tips to assist you in this endeavor.

# Step 2: Mount Google Drive

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 3: Build the CNN model

#Install TensorFlow Object Detection Dependencies

I installed the TensorFlow Object Detection API in this Google Colab instance. This requires cloning the TensorFlow models repository and running a couple installation commands. Click the play button to run the following sections of code.

In [46]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy


In [47]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Step 4: Train the model

In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/4559 Project/data',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')

model.fit(train_generator, epochs=10)


Found 5815 images belonging to 2 classes.
Epoch 1/10
182/182 [==============================] - 55s 286ms/step - loss: 0.4906 - accuracy: 0.7929
Epoch 2/10
182/182 [==============================] - 51s 281ms/step - loss: 0.3703 - accuracy: 0.8437
Epoch 3/10
182/182 [==============================] - 51s 279ms/step - loss: 0.3554 - accuracy: 0.8609
Epoch 4/10
182/182 [==============================] - 52s 285ms/step - loss: 0.3511 - accuracy: 0.8607
Epoch 5/10
182/182 [==============================] - 51s 282ms/step - loss: 0.3215 - accuracy: 0.8738
Epoch 6/10
182/182 [==============================] - 51s 277ms/step - loss: 0.2973 - accuracy: 0.8834
Epoch 7/10
182/182 [==============================] - 52s 284ms/step - loss: 0.2884 - accuracy: 0.8867
Epoch 8/10
182/182 [==============================] - 52s 283ms/step - loss: 0.2622 - accuracy: 0.8905
Epoch 9/10
182/182 [==============================] - 51s 280ms/step - loss: 0.2615 - accuracy: 0.8963
Epoch 10/10
182/182 [==========

# Step 5: Real-time face mask detection using OpenCV

In [49]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_face_mask(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (150, 150))
        face = np.expand_dims(face, axis=0)
        face = face / 255.0
        prediction = model.predict(face)

        if prediction > 0.5:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'With Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, 'Without Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    return frame

# open the webcam and start face mask detection

In [53]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = detect_face_mask(frame)
    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit
        break

cap.release()
cv2.destroyAllWindows()